In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [12]:
#loading the word index
word_index=imdb.get_word_index()
reverse_word_index={value:key for key,value in word_index.items()}

In [50]:
model=load_model('simplernn_imdb.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
model.get_weights()

[array([[ 0.03748291,  0.06203986,  0.02108498, ...,  0.03868888,
          0.05406481,  0.0193509 ],
        [-0.0287732 ,  0.03229147,  0.03093665, ..., -0.03648408,
          0.04531   ,  0.07698575],
        [ 0.01678771,  0.04975871,  0.05698651, ...,  0.04290387,
          0.05016747,  0.00834437],
        ...,
        [ 0.0407561 , -0.00393225,  0.05775513, ..., -0.00513427,
          0.01224387,  0.01241767],
        [ 0.01437132, -0.0136662 ,  0.06002134, ..., -0.05670445,
         -0.005495  ,  0.07244264],
        [-0.00689818, -0.09024904, -0.1428682 , ..., -0.02528787,
         -0.09744976,  0.09700393]], dtype=float32),
 array([[-0.04107316,  0.0682421 ,  0.03477223, ..., -0.19409819,
          0.05263697, -0.1354357 ],
        [ 0.05011497, -0.1434621 , -0.12283892, ...,  0.07670744,
         -0.0427107 ,  0.03824113],
        [ 0.13442245, -0.06724881,  0.09205511, ...,  0.06506746,
         -0.13638623, -0.0498581 ],
        ...,
        [ 0.05751491,  0.1376252 ,  0.0

In [54]:
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])
\
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# function to preprocess user input
def preprocess_text(text):
    import string
    
    # Clean the text: remove punctuation and convert to lowercase
    text_clean = text.lower().translate(str.maketrans('', '', string.punctuation))
    words = text_clean.split()
    
    # Convert words to indices
    encoded_review = []
    for word in words:
        if word in word_index:
            # IMDB word_index starts from 1, and the training data adds 3 to shift indices
            # This is because indices 0, 1, 2 are reserved for padding, start, unknown
            encoded_review.append(word_index[word] + 3)
        else:
            # Unknown word - use index 2 (as used in IMDB dataset)
            encoded_review.append(2)
    
    # Add start token (1) at the beginning to match training data format
    encoded_review = [1] + encoded_review
    
    # Pad sequence to maxlen=500 (same as training)
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [55]:
#prediction function
def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)
    # Set verbose=0 to suppress the progress bar output
    prediction=model.predict(preprocessed_input, verbose=0)
    sentiment='Positive' if prediction[0][0]>0.5 else 'Negative'
    return sentiment,prediction[0][0]
    

In [56]:
#example
example_review="This movie was fantastic! The plot was engaging and the acting was superb."
sentiment,score=predict_sentiment(example_review)
print(f'Review:{example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

Review:This movie was fantastic! The plot was engaging and the acting was superb.
Sentiment: Positive
Prediction Score: 0.532949686050415


In [57]:
# Test with more examples
test_reviews = [
    "This movie was terrible and boring. I hated it.",
    "Amazing film! Great acting and storyline.",
    "The worst movie I have ever seen. Complete waste of time.",
    "Excellent cinematography and outstanding performances.",
    "Not good at all. Very disappointing."
]

print("Testing multiple reviews:")
print("=" * 50)
for review in test_reviews:
    sentiment, score = predict_sentiment(review)
    print(f"Review: {review}")
    print(f"Sentiment: {sentiment}")
    print(f"Confidence Score: {score:.4f}")
    print("-" * 50)

Testing multiple reviews:
Review: This movie was terrible and boring. I hated it.
Sentiment: Negative
Confidence Score: 0.0119
--------------------------------------------------
Review: Amazing film! Great acting and storyline.
Sentiment: Positive
Confidence Score: 0.6343
--------------------------------------------------
Review: The worst movie I have ever seen. Complete waste of time.
Sentiment: Negative
Confidence Score: 0.0022
--------------------------------------------------
Review: Excellent cinematography and outstanding performances.
Sentiment: Positive
Confidence Score: 0.9443
--------------------------------------------------
Review: Not good at all. Very disappointing.
Sentiment: Negative
Confidence Score: 0.1402
--------------------------------------------------


In [37]:
# Interactive prediction function
def interactive_prediction():
    while True:
        user_input = input("\nEnter a movie review (or 'quit' to exit): ")
        if user_input.lower() == 'quit':
            break
        
        sentiment, score = predict_sentiment(user_input)
        print(f'\nReview: {user_input}')
        print(f'Sentiment: {sentiment}')
        print(f'Confidence Score: {score:.4f}')
        print('-' * 50)

# Uncomment the line below to run interactive mode
# interactive_prediction()

In [38]:
interactive_prediction()


Review: "Amazing film. Superb acting"
Sentiment: Positive
Confidence Score: 0.9005
--------------------------------------------------

Review: Great film
Sentiment: Positive
Confidence Score: 0.6159
--------------------------------------------------

Review: Great film
Sentiment: Positive
Confidence Score: 0.6159
--------------------------------------------------
